In [23]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from scipy.stats import gaussian_kde
import seaborn as sns
from joblib import Parallel, delayed
from IPython.display import clear_output

import warnings

warnings.filterwarnings('ignore')


1. RateConst Reader


In [24]:
def getAllRateInfo(file):   
    res=pd.DataFrame()
    try:
        temp=pd.read_csv(file)
        temp=temp.dropna()
        temp['Protein']=file.split('\\')[-1].replace('.RateConst.csv','')
        res=pd.concat([res,temp])
    except Exception as exp:
        return res
    
    return res
def getAllRate(data_path):
    results = Parallel(n_jobs=-1)(delayed(getAllRateInfo)(file) for file in [os.path.join(data_path,x) for x in os.listdir(data_path) if '.RateConst.csv' in x  ]) 
    all_data=pd.concat(results)
    all_data=all_data.reset_index(drop=True)
    all_data.columns=[x.strip() for x in all_data.columns]
    return all_data

2. Quant File Reader

In [25]:
def read_n_merge(skip,file):
    file_data=pd.read_csv(file,skiprows=skip,index_col=False)
    file_data.columns=[x.strip() for x in file_data.columns]
    isparsed=(set(['Peptide', 'UniqueToProtein', 'Exchangeable Hydrogens', 'Charge',
       'm/z(Sequence)', 'M0', 'M1', 'M2', 'M3', 'M4']).issubset(set(file_data.columns)))
    isparsed=isparsed and file_data.shape[0]>0    
    if isparsed:
        file_data['Protein']=file.split('\\')[-1].replace('.Quant.csv','') 
        return [isparsed,file_data]
    else:
        return [isparsed,None]
def get_df_all_quant_files(file):
    res=read_n_merge(1,file)
    all_data=None
    if res[0]: all_data=res[1]
    else: all_data=read_n_merge(3,file)[1]
    return all_data

def getquantfile(data_path):
    results = Parallel(n_jobs=-1)(delayed(get_df_all_quant_files)(file) for file in [os.path.join(data_path,x) for x in os.listdir(data_path) if '.Quant.csv' in x  ])
    all_data=pd.concat(results)
    all_data=all_data.reset_index(drop=True)
    all_data.columns=[x.strip() for x in all_data.columns]
    return all_data

3. Data source & path

In [26]:
organ="liver"
# organ="heart"
# organ="muscle"
data_path=r'C:\Workplace\Python\AnalysisForThePaper\NEH\d2ome_output\liverpool_liver'
# data_path=r'C:\Workplace\Python\AnalysisForThePaper\NEH\d2ome_output\liverpool_heart'
# data_path=r'H:\Warehouse\Data\DataUsedForPublication\Partial IsotopeProfile paper data used for publication\liverpool_CI\muscle'


data_quant=getquantfile(data_path)
data_rate=getAllRate(data_path)
data_rate=data_rate[(data_rate.Rsquared!=' -nan(ind)')&(data_rate.Rsquared!=' ')]



4. merge quant and rate files

In [27]:
merged=pd.merge(data_quant,
                data_rate,
                left_on=['Protein','Peptide','Charge'],
                right_on=['Protein','Peptides','Charge'])
merged=merged.reset_index(drop=True)
merged.shape

(18510, 219)

5. parameters

In [28]:
pw,ph=0.046,1.5574E-4
# rsquared= 0.99
maxrate=math.log(2)
exp_time=[0 ,1 ,2 ,3 ,6 ,7 ,9 ,13,16,21,24,31]

6. filter data

In [29]:
merged=merged[#(merged.Rsquared.astype('float')>=rsquared) &
              (merged.RateConstants< math.log(2))]
merged=merged.reset_index(drop=True)
merged.shape

(17344, 219)

7. Helper functions

In [30]:
def get_I0_t(I0_0,I0_asymp,k,t):
    return I0_asymp + (I0_0-I0_asymp)*math.exp(-k*t)

def get_I0_asmyp(I0_0,neh):
    return I0_0*( (1 - pw/(1-ph))**neh )

def get_I0_exp(index,suffix):
    _sum=float(float(merged.loc[index,f'I0{suffix}'])+float(merged.loc[index,f'I1{suffix}'])+float(merged.loc[index,f'I2{suffix}'])+float(merged.loc[index,f'I3{suffix}'])+
                        float(merged.loc[index,f'I4{suffix}'])+float(merged.loc[index,f'I5{suffix}']))
    if _sum==0: return None
    else: return float(merged.loc[index,f'I0{suffix}'])/_sum

def getNewKestimate(M0_0,I0_0_exp,I0_asymp,t,I0_t_exp,numberofTerms): 
        
    base=(I0_0_exp-I0_t_exp)/(M0_0-I0_asymp)    
    new_kt= sum([ (base**i)/i for i in range(1,numberofTerms+1)])     
    new_k=new_kt/t    
    return new_k


def getNewKestimateByIndex(time,numberofterms,index):
        
        time_index=exp_time.index(time)
        suffix=f".{time_index}"  
        # print('time',time,time_index)
                                              
        if merged.loc[index,f'I0{suffix}'] == ' ' or merged.loc[index,f'I0'] == ' ': 
            return None            
        
        I0_0_exp= get_I0_exp(index,"")
        I0_t_exp=get_I0_exp(index,suffix)
        
        if I0_0_exp == None or I0_t_exp ==None or  I0_t_exp >  I0_0_exp: 
            return None

        M0_0=float(merged.loc[index,f'M0'])/100
        k=float(merged.loc[index,f'RateConstants'])
        I0_asymp=M0_0*( (1 - pw/(1-ph))**merged.loc[index,f'Exchangeable Hydrogens'] ) 
        i0_t_theo= I0_asymp + (M0_0-I0_asymp)*math.exp(-k*time)     
        
        exp_fsr= (I0_0_exp-I0_t_exp)/(M0_0-I0_asymp)
        # print(exp_fsr, (I0_0_exp-I0_t_exp),(M0_0-I0_asymp),"==>",(I0_0_exp,I0_t_exp),(M0_0,I0_asymp) )
        if exp_fsr > 1: return None
        exp_kt= -math.log( 1 - exp_fsr) 
        d2ome_kt=k*time
        
        error=   abs(exp_kt - d2ome_kt)/d2ome_kt

        return[merged.loc[index,'Protein'],
               merged.loc[index,'Peptide'],
               merged.loc[index,'Charge'],                
                    merged.loc[index,f'Exchangeable Hydrogens'],
                    M0_0,I0_0_exp,
                    I0_asymp,
                    I0_t_exp,i0_t_theo,k] +[getNewKestimate(M0_0,I0_0_exp,I0_asymp,time,I0_t_exp,numberofterms),error]
        
def computeAllNewRates(_time,_numberOfTerms):
    # res = Parallel(n_jobs=-1)(delayed(getNewKestimateByIndex)(_time,_numberOfTerms,index) for index in range(merged.shape[0]))
    res=[getNewKestimateByIndex(_time,_numberOfTerms,index) for index in range(merged.shape[0])]
    res=pd.DataFrame([r for r in res if r!=None])
    res.columns=['Protein','Peptide','Charge',"NEH","M0","I0_0_exp","I0_asmp",'i0_t_exp','i0_t_theo','d2ome_k','new_k','error']
    return res

def plot_hitmap(x,y,time,path="",title=""):
          
    plt.rc('font',size=14)
    plt.figure(figsize=(7,6))
    plt.set_cmap('jet')  
    
    x=np.array(x)
    y=np.array(y)    
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    r=np.corrcoef(x,y)
    
    # Create the main plot
    _, ax1 = plt.subplots()


    cb=ax1.scatter(x, y, c=z, s=10,alpha=0.8)  
    ax1.plot(x,x,ls='-.',c='k',linewidth=1,alpha=0.5,label=f"n={len(list(x))}\nr={int(1000*r[0][1])/1000}")      
    cbar = plt.colorbar(cb)
    ax1.set_ylabel("k (Single time point)")
    ax1.set_xlabel("k (Time course data)")    
    ax1.legend()  
    # Remove top and right spines
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    # ax1.set_xlim([0,min(max(x),0.3)])
    # ax1.set_ylim([0,min(max(y),0.3)])
    
    
    # Create the second x-axis
    ax2 = ax1.twiny()
    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks([(i*time) for i in ax1.get_xticks()])
    ax2.set_xlabel('kt', color='k')
    ax2.spines['right'].set_visible(False)


    # Adjust layout to prevent labels from overlapping
    plt.tight_layout()
    if len(path)>0:
        plt.savefig(f"{path}/{title}.jpeg",dpi=900)
        
    plt.show()
    
    
   
def plotRateRDDensity(res,path="",title=""):
    plt.rc('font',size=14)
    plt.figure(figsize=(8,6))
    rd_k= (res.d2ome_k - res.new_k)/res.d2ome_k
    ax=sns.histplot(rd_k, kde=True, #bins=50,
                label=f"n={len(rd_k)},μ = {int(100*np.mean(rd_k))/100}, \nmedian = {int(100*np.median(rd_k))/100}, \nsd = {int(100*np.std(rd_k))/100}",
                color='white',alpha=0.1,edgecolor=('black',0.75) )
    ax.lines[0].set_color('k')
    ax.lines[0].set_alpha(0.75) 
    ax.lines[0].set_ls(':') 
    
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xlabel(r'$\frac{ k _{\:Time\: course\: data} - k _{\:Single\: time\: point}}{k _{\:Time\: course\: data}}$')
    plt.legend()
    plt.tight_layout()
    if len(path)>0:
        plt.savefig(f"{path}/{title}_RD_k.jpeg",dpi=900)
    plt.show()

8. Prepare all new k values

In [31]:
# _res=computeAllNewRates(1,1)
# plot_hitmap(_res.d2ome_k,_res.new_k)
# plotRateRDDensity(_res)
# plotI0_expRDDensity(_res)
# plotSubgroup(_res,"","",1)

In [32]:
allmerged=merged.copy()
allmerged.shape

(17344, 219)

In [33]:
# for _time in tqdm([1 ,2 ,3 ,6 ,7,16]):
for _time in tqdm([6 ,7,16]):    
    for _rsquared in [0.99,0.95,0.9][::-1]:
        for _numberofterms in (1,2,3,6,10):
                        
            try:                            
                merged=allmerged.copy()
                merged=merged[(merged.Rsquared.astype('float')>=_rsquared)]
                merged=merged.reset_index(drop=True)

                _res=computeAllNewRates(_time,_numberofterms)

                _res=_res[(_res.d2ome_k.astype('float')>=0)]
                _res=_res[(_res.new_k.astype('float')>=0)]

                _path1=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/"
                _path2=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/DenseRegion/"
                _path3=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/10Percent_K_RD/"
                _paths=[_path1,_path2,_path3]
                for _path in _paths:
                    if not os.path.exists(_path): os.makedirs(_path)
                    break

                _title=f"{organ}_day_{_time}_numberofterms_{_numberofterms}"    


                _res["i0_d2ome"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.d2ome_k[i]*_time) for i in range(_res.shape[0])]
                _res["i0_new"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.new_k[i]*_time) for i in range(_res.shape[0])]

                _res["fsr_d2ome"]=[ (_res.I0_0_exp[i] - _res.i0_d2ome[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
                _res["fsr_new"]=[ (_res.I0_0_exp[i] - _res.i0_new[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
                _res["fsr_exp"]=[ (_res.I0_0_exp[i] - _res.i0_t_exp[i] )/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]

                _res['rd_fsr']= (_res.fsr_exp-_res.fsr_new)/_res.fsr_new


                _res=_res[abs(_res['rd_fsr'] )<0.03]
                _res=_res.reset_index(drop=True)


                plt.scatter(_res.fsr_exp,_res.fsr_new,c='k',alpha=0.4)
                plt.plot(_res.fsr_exp,_res.fsr_exp,c='r')
                plt.xlabel('FSR (exp)')
                plt.ylabel('FSR (approx.)')
                plt.savefig(f"{_path}/{_title}_fsr.jpeg",dpi=900)
                plt.show()

                sns.histplot(_res['rd_fsr'] )
                plt.xlabel("Relative Difference (FSR)")


                plot_hitmap(_res.d2ome_k,_res.new_k,_time,_path1,_title)

                plotRateRDDensity(_res,_path1,_title)
                clear_output(wait=True)  
            except:
                continue


100%|██████████| 3/3 [14:36<00:00, 292.12s/it]


In [34]:

# _time=1
# _rsquared=0.95
# _numberofterms=10

# _path1=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/"
# _path2=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/DenseRegion/"
# _path3=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/10Percent_K_RD/"
# _paths=[_path1,_path2,_path3]
# for _path in _paths:
#     if not os.path.exists(_path): os.makedirs(_path)

            
# merged=allmerged.copy()
# merged=merged[(merged.Rsquared.astype('float')>=_rsquared)]
# merged=merged.reset_index(drop=True)

# _res=computeAllNewRates(_time,_numberofterms)

# _res=_res[(_res.d2ome_k.astype('float')>=0)]
# _res=_res[(_res.new_k.astype('float')>=0)]

# _path1=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/"
# _path2=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/DenseRegion/"
# _path3=f"Figures/{organ}/Day_{_time}/r2_{_rsquared}/numberofterms_{_numberofterms}/subgroup/10Percent_K_RD/"
# _paths=[_path1,_path2,_path3]
# for _path in _paths:
#     if not os.path.exists(_path): os.makedirs(_path)
#     break

# _title=f"{organ}_day_{_time}_numberofterms_{_numberofterms}"    


# _res["i0_d2ome"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.d2ome_k[i]*_time) for i in range(_res.shape[0])]
# _res["i0_new"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.new_k[i]*_time) for i in range(_res.shape[0])]

# _res["fsr_d2ome"]=[ (_res.I0_0_exp[i] - _res.i0_d2ome[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
# _res["fsr_new"]=[ (_res.I0_0_exp[i] - _res.i0_new[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
# _res["fsr_exp"]=[ (_res.I0_0_exp[i] - _res.i0_t_exp[i] )/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]

# _res['rd_fsr']= (_res.fsr_exp-_res.fsr_new)/_res.fsr_new


# _res=_res[abs(_res['rd_fsr'] )<0.03]
# _res=_res.reset_index(drop=True)


# plt.scatter(_res.fsr_exp,_res.fsr_new,c='k',alpha=0.4)
# plt.plot(_res.fsr_exp,_res.fsr_exp,c='r')
# plt.xlabel('FSR (exp)')
# plt.ylabel('FSR (approx.)')
# plt.savefig(f"{_path}/{_title}_fsr.jpeg",dpi=900)
# plt.show()

# sns.histplot(_res['rd_fsr'] )
# plt.xlabel("Relative Difference (FSR)")


# plot_hitmap(_res.d2ome_k,_res.new_k,_time,_path1,_title)

# plotRateRDDensity(_res,_path1,_title)



In [35]:
# temp=_res.sort_values(by='error')
# temp=temp[temp.error<=0.2]
# plot_hitmap(temp.d2ome_k,temp.new_k,_time,_path1,_title)

In [36]:
# _res.head(5)

In [37]:
# _res["i0_d2ome"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.d2ome_k[i]*_time) for i in range(_res.shape[0])]
# _res["i0_new"]=[_res.I0_asmp[i] + (_res.I0_0_exp[i] - _res.I0_asmp[i])*math.exp(-_res.new_k[i]*_time) for i in range(_res.shape[0])]

# _res["fsr_d2ome"]=[ (_res.I0_0_exp[i] - _res.i0_d2ome[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
# _res["fsr_new"]=[ (_res.I0_0_exp[i] - _res.i0_new[i])/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]
# _res["fsr_exp"]=[ (_res.I0_0_exp[i] - _res.i0_t_exp[i] )/(_res.M0[i] - _res.I0_asmp[i])  for i in range(_res.shape[0]) ]

# _res['rd_fsr']= (_res.fsr_exp-_res.fsr_new)/_res.fsr_new


In [38]:

# _res=_res[abs(_res['rd_fsr'] )<0.03]
# _res=_res.reset_index(drop=True)


# # plt.scatter(_res.i0_t_exp,y)
# # plt.plot(_res.i0_t_exp,_res.i0_t_exp,c='r')

# # plt.scatter(_res.fsr_exp,_res.fsr_d2ome)
# plt.scatter(_res.fsr_exp,_res.fsr_new,c='k',alpha=0.4)
# plt.plot(_res.fsr_exp,_res.fsr_exp,c='r')
# plt.xlabel('FSR (exp)')
# plt.ylabel('FSR (approx.)')
# plt.savefig(f"{_path}/{_title}_fsr.jpeg",dpi=900)
# plt.show()

# sns.histplot(_res['rd_fsr'] )
# plt.xlabel("Relative Difference (FSR)")


# plot_hitmap(_res.d2ome_k,_res.new_k,_time,_path1,_title)

# plotRateRDDensity(_res,_path1,_title)

In [39]:
# plt.scatter(_res.d2ome_k,_res.new_k)

In [40]:
# plt.plot(
#     [-math.log(1-i) for i in ((_res.I0_0_exp - _res.i0_t_exp)/(_res.M0 - _res.I0_asmp))],
#     [-math.log(1-i) for i in ((_res.I0_0_exp - _res.i0_t_exp)/(_res.M0 - _res.I0_asmp))]
# )
# plt.scatter(  
#             [-math.log(1-i) for i in ((_res.I0_0_exp - _res.i0_t_exp)/(_res.M0 - _res.I0_asmp))],
#             (_res.new_k*_time)
#             )

In [41]:
# plt.scatter( (_res.new_k*_time),((_res.I0_0_exp - _res.i0_t_exp)/(_res.M0 - _res.I0_asmp)) )

In [42]:
# _res